In this notebook i try with differents parameters Xception
 

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras import optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import tensorflow as tf

Using TensorFlow backend.


In [0]:
%cd /content/drive/My Drive/deeplearning/2020.deeplearning/U3project/food

/content/drive/My Drive/deeplearning/2020.deeplearning/U3project/food


In [0]:
train_data_dir='prueba'

In [0]:
TEST_SPLIT = 0.25
epochs = 80
#batch_size = 16
batch_size=16
MICRO_DATA = True # very small subset (just 3 groups)
SAMPLE_TRAINING = False # make train set smaller for faster iteration
IMG_SIZE = (112, 112)
LEARNING_RATE = 3e-2
seed=1

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', seed=seed) # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', seed=seed) # set as validation data

Found 8037 images belonging to 10 classes.
Found 2009 images belonging to 10 classes.


#8 clases:  
pancakes/     peking_duck/  pizza/      poutine/
  panna_cotta/  pho/          pork_chop/  prime_rib/

In [0]:
model_final=tf.keras.models.load_model('vgg_finetuned_eightclasses.model')

In [0]:
model_final=tf.keras.models.load_model('inception_finetuned_eightclasses.model')

inception 8 clases


In [0]:
#p_pr=model_final.predict(validation_generator)
loss, acc = model_final.evaluate_generator(validation_generator, 400)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Restored model, accuracy: 60.42%


para 10 clases mussels/  pancakes/     peking_duck/  pizza/      poutine/
nachos/   panna_cotta/  pho/          pork_chop/  prime_rib/

In [0]:
model_final=tf.keras.models.load_model('vgg_finetuned_tenclasses.model')

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', seed=seed) # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', seed=seed) # set as validation data

Found 8037 images belonging to 10 classes.
Found 2009 images belonging to 10 classes.


In [0]:
#p_pr=model_final.predict(validation_generator)
loss, acc = model_final.evaluate_generator(validation_generator, 400)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Instructions for updating:
Please use Model.evaluate, which supports generators.
Restored model, accuracy: 59.12%


Xception

In [0]:
model_final=tf.keras.models.load_model('Xception_finetuned_tenclasses.model')

In [0]:
loss, acc = model_final.evaluate_generator(validation_generator, 400)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Restored model, accuracy: 68.93%


In [0]:
model_final=tf.keras.models.load_model('Xception_finetuned_tenclasses')

In [0]:
loss, acc = model_final.evaluate_generator(validation_generator, 400)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Restored model, accuracy: 76.49%


In [0]:
prob=model_final.predict(validation_generator)
import numpy as np
ypred=np.argmax(prob,axis=1)

y_true=validation_generator.classes

tf.math.confusion_matrix(y_true,ypred)

#grupo 1 

fried_rice/    gnocchi/      grilled_cheese_sandwich/
garlic_bread/  greek_salad/  grilled_salmon/

In [0]:
train_data_dir='grupo1'

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', seed=seed) # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', seed=seed) # set as validation data

Found 4825 images belonging to 6 classes.
Found 1206 images belonging to 6 classes.


In [0]:
#model_final.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.00008, beta_1=0.9, beta_2=0.97, epsilon=1e-7))
#batch_size=128
from keras import applications
model =applications.Xception(weights = "imagenet", include_top=False, input_shape = (112,112,3))
for layer in model.layers[:100]:
    layer.trainable = False
regparameter=0.001
#Adding custom Layers 

x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=regparameter,l2=regparameter))(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation="relu")(x)
predictions = Dense(6, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
model_final.summary()

In [0]:
chkpt = ModelCheckpoint(filepath='grupo1_6class_reg_001.hdf5', save_best_only=True, save_weights_only=True)

In [0]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 20,callbacks=[chkpt])

Epoch 1/20
301/301 [==============================] - 3841s 13s/step - loss: 4.5706 - accuracy: 0.5176 - val_loss: 1.8842 - val_accuracy: 0.6358
Epoch 2/20
301/301 [==============================] - 681s 2s/step - loss: 2.3585 - accuracy: 0.6675 - val_loss: 2.4582 - val_accuracy: 0.5891
Epoch 3/20
301/301 [==============================] - 680s 2s/step - loss: 2.0176 - accuracy: 0.7440 - val_loss: 2.2309 - val_accuracy: 0.6639
Epoch 4/20
301/301 [==============================] - 679s 2s/step - loss: 1.8150 - accuracy: 0.7898 - val_loss: 1.9290 - val_accuracy: 0.6353
Epoch 5/20
301/301 [==============================] - 687s 2s/step - loss: 1.9702 - accuracy: 0.8201 - val_loss: 2.0777 - val_accuracy: 0.6160
Epoch 6/20
301/301 [==============================] - 655s 2s/step - loss: 1.5468 - accuracy: 0.8659 - val_loss: 3.0738 - val_accuracy: 0.6387
Epoch 7/20
301/301 [==============================] - 671s 2s/step - loss: 1.4685 - accuracy: 0.8896 - val_loss: 2.7008 - val_accuracy: 0.67

In [0]:
prob=model_final.predict(validation_generator)
import numpy as np
ypred=np.argmax(prob,axis=1)

y_true=validation_generator.classes

tf.math.confusion_matrix(y_true,ypred)

#Grupo 2 


In [0]:
train_data_dir='grupo2'

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', seed=seed) # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', seed=seed) # set as validation data

Found 4831 images belonging to 6 classes.
Found 1207 images belonging to 6 classes.


In [0]:
#model_final.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.00008, beta_1=0.9, beta_2=0.97, epsilon=1e-7))
#batch_size=128
from keras import applications
model =applications.Xception(weights = "imagenet", include_top=False, input_shape = (112,112,3))
for layer in model.layers[:100]:
    layer.trainable = False
regparameter=0.001
#Adding custom Layers 

x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=regparameter,l2=regparameter))(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation="relu")(x)
predictions = Dense(6, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
model_final.summary()

In [0]:
chkpt = ModelCheckpoint(filepath='grupo2_6class_reg_001.hdf5', save_best_only=True, save_weights_only=True)

In [0]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 20,callbacks=[chkpt])

In [0]:
model_final.load_weights('grupo2_6class_reg_001.hdf5')


In [0]:
loss,acc = model_final.evaluate(validation_generator, verbose=1)


76/76 [==============================] - 542s 7s/step


In [0]:
print(loss,acc)

2.5716795921325684 0.7257663607597351


In [0]:
prob=model_final.predict(validation_generator, verbose=1))
import numpy as np
ypred=np.argmax(prob,axis=1)

y_true=validation_generator.classes

tf.math.confusion_matrix(y_true,ypred)

In [0]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 50,callbacks=[chkpt])

#Grupo 3


In [0]:
train_data_dir='grupo3'

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', seed=seed) # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', seed=seed) # set as validation data

Found 4814 images belonging to 6 classes.
Found 1203 images belonging to 6 classes.


In [0]:
#model_final.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.00008, beta_1=0.9, beta_2=0.97, epsilon=1e-7))
#batch_size=128
from keras import applications
model =applications.Xception(weights = "imagenet", include_top=False, input_shape = (112,112,3))
for layer in model.layers[:100]:
    layer.trainable = False
regparameter=0.001
#Adding custom Layers 

x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=regparameter,l2=regparameter))(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation="relu")(x)
predictions = Dense(6, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
model_final.summary()

In [0]:
chkpt = ModelCheckpoint(filepath='grupo3_6class_reg_001.hdf5', save_best_only=True, save_weights_only=True)

In [0]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 20,callbacks=[chkpt])

In [0]:
model_final.load_weights('grupo3_6class_reg_001.hdf5')



In [0]:
loss,acc = model_final.evaluate(validation_generator, verbose=1)


76/76 [==============================] - 82s 1s/step


In [0]:
print(loss,acc)

1.032043695449829 0.6824604868888855
